# TODO Markdown title here and explaination

In [1]:
import re
import os.path
import uritools
import numpy as np
import scipy as sp
import pandas as pd
from bs4 import BeautifulSoup

import seaborn as sns
import matplotlib.pyplot as plt

from pyspark.sql import *
import pyspark.sql.functions as f # wierd that I have to do that

In [2]:
# General parameters
%matplotlib inline
plt.style.use('seaborn')#switch to seaborn style
plt.rcParams["figure.figsize"] = [16,10]

spark = SparkSession.builder.getOrCreate()

DATA_FOLDER = './data/'
RECIPES_PATH = DATA_FOLDER + 'recipePages/'

In [10]:
# TODO: for loop
html_string = RECIPES_PATH + '0a7e6e2cae6d4da800d13ef59e760dd3.html'
html = open(html_string,'r')
soup = BeautifulSoup(open(html_string), 'html.parser')
re.sub(r'\n|\t', '',soup.title.text)

'Banana Muffins I Recipe - Allrecipes.com'

In [11]:
# Title of the recipe, TODO remove the website name?
re.sub(r'\n|\t', '',soup.title.text)

'Banana Muffins I Recipe - Allrecipes.com'

In [135]:
# def scrap_AllRecipe_com( soup )
# Get All the ingredients
ingredient_list = list()
for tag in soup.findAll("li", {"class": re.compile('.*ingredient*', flags=re.IGNORECASE)}):
    ingredient_list.append(re.sub(r'\n|  ', '', tag.text))
            
# Printing  ingredients
print("\nIngredients:",ingredient_list)

## How to append to list?
# If 'banana' doesnt exist in the list -> add mashed ripe banana to the list
# if afterwards we see banana, by regex matching we should get back mashed ripe banana, in this case we rename
# the mashed ripe to banana ( comparing the size of the keywords)

# Fetch nutritional information
# TODO, we can get WAY MORE if you look just below in the website
# how can we store all the informations if somes are missing?
nutritive_info = soup.findAll(True, {"id": re.compile('.*nutri*.', flags=re.IGNORECASE)})

# Sub Extracting nutritive informations per Serving
soup_nutrition = BeautifulSoup(str(nutritive_info), 'html.parser')

# Awesome!!!! TODO
nutrition_dict = {}
for tag in soup_nutrition.findAll('span'):
    nutrition_dict.update({tag.get("class")[0]: tag.text})

# Printing nutrition
print("\nNutrition:", nutrition_dict)

# Ratings
rating = re.search(r'AverageRating":(\d+\.\d+)',soup.text, re.IGNORECASE).group(1)
n_rating = soup.findAll("span", {"class": "count"})[0].text

print("\nAverage ratings:", rating, "Number of vote:", n_rating)

# return ingredient_list, nutrition_dict, rating


Ingredients: ['1 cup all-purpose flour', '1 tablespoon baking powder', '1/2 teaspoon baking soda', '1/4 teaspoon salt', '1 cup mashed ripe banana', '1/4 cup white sugar', '1/4 cup sour cream', '1 egg', '1/2 teaspoon vanilla extract']

Nutrition: {'calories': '88', 'fat': '1.6g', 'cholesterol': '20mg', 'sodium': '194mg', 'totalcarbs': '16.9g', 'dietaryfiber': '0.8g', 'protein': '2g'}

Average ratings: 4.3 Number of vote: 263


In [ ]:
# Flow chart
# CHeck every website name
# create for each website a dedicated function to fetch informations
# Keep in mind to avoid empty recipes website ( as index pages)
# save all this to a pysprk parquet file 

In [ ]:
# For each file we first look at every possiblr website
website_titles= {}

files = os.listdir(RECIPES_PATH)
for filename in files:
    
    # Opening HTML file
    with open(RECIPES_PATH + filename, 'rb') as file:
        try:
            # Parsing the HTML file
            soup = BeautifulSoup(file, 'html.parser')
    
            # Retrieve the HostName
            hostname = uritools.urisplit(soup.find('a',href=True)['href']).gethost()
    
            # Append the HostName if not seen yet
            website_titles.update({hostname: filename})
            
        except : # whatever reader errors you care about
            continue

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
